# Guided Project: Answering Business Questions using SQL

We will answer some business questions using SQL. For datasource we have Chinook SQLite database with music store data (tracks, invoices, etc.).

<img src="https://s3.amazonaws.com/dq-content/191/chinook-schema.svg", width=600px>

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

Let's inspect what tables and views we actually have in our database and build some basic queries.

In [2]:
%%sql
select type, name from sqlite_master where type in ('table', 'view')

Done.


type,name
table,album
table,artist
table,customer
table,employee
table,genre
table,invoice
table,invoice_line
table,media_type
table,playlist
table,playlist_track


In [3]:
%%sql
select
     tr.track_id
    ,tr.name as track_name
    ,mt.name as media_type_name
    ,g.name as genre_name
    ,al.title as album_title
    ,ar.name as artist_name
from track tr
    join media_type mt on mt.media_type_id = tr.media_type_id
    join genre g on g.genre_id = tr.genre_id
    join album al on al.album_id = tr.album_id
    join artist ar on ar.artist_id = al.artist_id
limit 10

Done.


track_id,track_name,media_type_name,genre_name,album_title,artist_name
1,For Those About To Rock (We Salute You),MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
2,Balls to the Wall,Protected AAC audio file,Rock,Balls to the Wall,Accept
3,Fast As a Shark,Protected AAC audio file,Rock,Restless and Wild,Accept
4,Restless and Wild,Protected AAC audio file,Rock,Restless and Wild,Accept
5,Princess of the Dawn,Protected AAC audio file,Rock,Restless and Wild,Accept
6,Put The Finger On You,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
7,Let's Get It Up,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
8,Inject The Venom,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
9,Snowballed,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
10,Evil Walks,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC


In [4]:
%%sql
select
    cus.company as customer_company
    ,sum(il.quantity) as songs_sold
    ,round(sum(il.unit_price * il.quantity), 2) as invoice_amount
from invoice_line il
    join invoice i on i.invoice_id = il.invoice_id
    join customer cus on cus.customer_id = i.customer_id
group by cus.company
order by 3 desc
limit 10

Done.


customer_company,songs_sold,invoice_amount
None,3961,3921.39
JetBrains s.r.o.,146,144.54
Embraer - Empresa Brasileira de Aeronáutica S.A.,110,108.9
Microsoft Corporation,99,98.01
Riotur,83,82.17
Google Inc.,75,74.25
Banco do Brasil S.A.,70,69.3
Rogers Canada,67,66.33
Woodstock Discos,61,60.39
Apple Inc.,55,54.45


## Selecting Albums to Purchase

Business needs to know the most profitable music genres, and if we have any recommendations for Hip-Hop, Punk, Pop or Blues genres.

In [5]:
%%sql
with genre_purchases as (
select
    g.name as genre_name
    ,sum(il.quantity) as track_quantity
    ,sum(i.total) as invoice_amount
from invoice_line il
    join invoice i on i.invoice_id = il.invoice_id
    join track tr on tr.track_id = il.track_id
    join genre g on g.genre_id = tr.genre_id
where i.billing_country = 'USA'
group by g.name
)

select
    gp.genre_name
    ,gp.track_quantity
    ,round(gp.invoice_amount, 2) as invoice_amount
    ,round(cast(gp.track_quantity as float) / tot.total_track_quantity, 4) as track_quantity_pct
    ,round(cast(gp.invoice_amount as float) / tot.total_invoice_amount, 4) as invoice_amount_pct
from genre_purchases gp
    cross join (
        select
            sum(track_quantity) as total_track_quantity
            ,sum(invoice_amount) as total_invoice_amount
        from genre_purchases
    ) tot
order by 5 desc

Done.


genre_name,track_quantity,invoice_amount,track_quantity_pct,invoice_amount_pct
Rock,561,5568.75,0.5338,0.5352
Alternative & Punk,130,1234.53,0.1237,0.1186
Metal,124,1025.64,0.118,0.0986
R&B/Soul,53,633.6,0.0504,0.0609
Blues,36,453.42,0.0343,0.0436
Alternative,35,369.27,0.0333,0.0355
Hip Hop/Rap,20,366.3,0.019,0.0352
Pop,22,201.96,0.0209,0.0194
Latin,22,162.36,0.0209,0.0156
Jazz,14,125.73,0.0133,0.0121


Punk (11%) comes second in the list right after the **Rock** (53%) compositions and could be a great opportunity to make good revenue. **Blues** (4%), **Hip-Hop** (3%) and **Pop** (2%) are not so popular genres, and it depends how much money we have got from these artists.

## Analyzing Employee Sales Performance

What is the total dollar amount of sales assigned to each sales support agent within the company?

In [6]:
%%sql
select
    e.first_name || " " || e.last_name as employee_name
    ,e.title as employee_title
    ,e.country
    ,e.state
    ,e.city
    ,e.hire_date
    ,2020 - strftime("%Y", e.birthdate) as age
    ,round(sum(i.total), 2) as invoice_amount
from invoice i
    join customer c on c.customer_id = i.customer_id
    join employee e on e.employee_id = c.support_rep_id
group by e.first_name, e.last_name, e.title, e.country, e.state, e.city, e.hire_date, e.birthdate
order by sum(i.total) desc

Done.


employee_name,employee_title,country,state,city,hire_date,age,invoice_amount
Jane Peacock,Sales Support Agent,Canada,AB,Calgary,2017-04-01 00:00:00,47,1731.51
Margaret Park,Sales Support Agent,Canada,AB,Calgary,2017-05-03 00:00:00,73,1584.0
Steve Johnson,Sales Support Agent,Canada,AB,Calgary,2017-10-17 00:00:00,55,1393.92


In [7]:
%%sql
-- Double check just to make sure amount is correct
select
    1731.51 + 1584 + 1393.92
    ,round(sum(total), 2)
from invoice

Done.


1731.51 + 1584 + 1393.92,"round(sum(total), 2)"
4709.43,4709.43


While there is no correlation between employee age and invoice amount, the difference may occur due the hire date. The longer an employee works, the more money he brings.

## Analyzing Sales by Country

Collate data on purchases from different countries. A country which has only one customer should be grouped as "Other".

In [8]:
%%sql
select
    country_group
    ,number_of_customers
    ,number_of_invoices
    ,round(invoice_amount, 2) as invoice_amount
    ,round(cast(invoice_amount as float) / number_of_customers, 2) as avg_customer_amount
    ,round(cast(invoice_amount as float) / number_of_invoices, 2) as avg_invoice_amount
from (
    select
        cgr.country_group
        ,case when cgr.country_group = 'Other' then 1 else 0 end as country_sort
        ,count(distinct i.customer_id) as number_of_customers
        ,count(*) as number_of_invoices
        ,sum(i.total) as invoice_amount
    from invoice i
        inner join (
            select
                billing_country
                ,case when count(distinct customer_id) = 1 then 'Other'
                    else billing_country end as country_group
            from invoice
            group by billing_country        
        ) cgr on cgr.billing_country = i.billing_country
    group by cgr.country_group
) c
order by country_sort, invoice_amount desc

Done.


country_group,number_of_customers,number_of_invoices,invoice_amount,avg_customer_amount,avg_invoice_amount
USA,13,131,1040.49,80.04,7.94
Canada,8,76,535.59,66.95,7.05
Brazil,5,61,427.68,85.54,7.01
France,5,50,389.07,77.81,7.78
Germany,4,41,334.62,83.66,8.16
Czech Republic,2,30,273.24,136.62,9.11
United Kingdom,3,28,245.52,81.84,8.77
Portugal,2,29,185.13,92.57,6.38
India,2,21,183.15,91.57,8.72
Other,15,147,1094.94,73.0,7.45


Despite the small number of customers, the following countries have largest average amounts per both order and customer:
* **Czech Republic** (2 customers, 136.62\$ / Customer, 9.11\$ / Order)
* **United Kingdom** (3 customers, 81.84\$ / Customer, 8.77\$ / Order)
* **India** (2 customers, 91.57\$ / Customer, 8.72\$ / Invoice)  

## Albums vs Individual Tracks 

Categorize each invoice as either an album purchase or not, and calculate number of invoices and percentage of invoices. This is a tricky one, because we does not know how to handle invoice with full album plus several songs from another album. We will provide two different definitions:

* In the first one we will define "album purchase" as following: An invoice containing full albums only.
* The second one: An invoice containing at least one full album.

In [9]:
%%sql
drop view if exists invoice_albums;
create view invoice_albums as
    
with compare as (
    select
        ia.invoice_id
        ,ia.album_id
        ,min(itr.track_id is not null) as full_album
    from (
        select distinct il.invoice_id, tr.album_id
        from invoice_line il join track tr on tr.track_id = il.track_id
    ) ia
        left join track tr on tr.album_id = ia.album_id
        left join invoice_line itr on itr.invoice_id = ia.invoice_id
            and itr.track_id = tr.track_id
    group by ia.invoice_id, ia.album_id
)

select
    invoice_id
    ,count(*) as total_albums
    ,sum(full_album) as full_albums
from compare
group by invoice_id

Done.
Done.


[]

The query with invoices containing full albums only.

In [10]:
%%sql
select
    case
        when full_albums > 0 and total_albums = full_albums
            then 'Full Album' else 'Non Album' end as invoice_type
    ,count(*) as invoices
    ,round(cast(count(*) as float) / (select count(*) from invoice), 2) as invoices_pct
from invoice_albums
group by 1

Done.


invoice_type,invoices,invoices_pct
Full Album,114,0.19
Non Album,500,0.81


The query with invoices containing at least one full album.

In [11]:
%%sql
select
    case
        when full_albums > 0 then 'Full Album' else 'Non Album' end as invoice_type
    ,count(*) as invoices
    ,round(cast(count(*) as float) / (select count(*) from invoice), 2) as invoices_pct
from invoice_albums
group by 1

Done.


invoice_type,invoices,invoices_pct
Full Album,201,0.33
Non Album,413,0.67


Business is currently considering to purchase only the most popular tracks from each album, instead of purchasing every track from an album. This could lead to losing from **20% to 30%** of revenue.

In [12]:
%%sql -- query using 'except' statement (less optimized due the selects in select what is slow af)

with missing as (
    select
        invoice_id
        ,(
            select count(*)
            from (
                -- All album tracks
                select tr.track_id from track tr
                where tr.album_id = ifa.first_album
                
                except
                
                -- All invoiced tracks
                select il.track_id from invoice_line il
                where il.invoice_id = ifa.invoice_id
            ) al_diff
        ) as missing_tracks
    from (
        select 
            il.invoice_id
            ,min(tr.album_id) as first_album
        from invoice_line il join track tr on tr.track_id = il.track_id
        group by il.invoice_id
    ) ifa
)

select
    case when missing_tracks = 0 then 'Full Album' else 'Non Album' end as invoice_type
    ,count(*) as invoices
    ,round(cast(count(*) as float) / (select count(*) from invoice), 2) as invoices_pct
from missing
group by 1

Done.


invoice_type,invoices,invoices_pct
Full Album,117,0.19
Non Album,497,0.81
